In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!ls ../mumu/MuMu_dataset/

amazon_metadata_MuMu.json  MuMu_dataset_multi-label.csv
amazon_reviews_MuMu.json   MuMu_dataset_single-label.csv


In [3]:
import json

mumu_dir = '../mumu/MuMu_dataset/'

db_review = []
with open(mumu_dir + '/amazon_reviews_MuMu.json') as f:
    for row in f.readlines():
        db_review.append(json.loads(row))

len(db_review)

447583

In [4]:
import csv

msd_id_map = {}
count = 0


with open(mumu_dir + '/MuMu_dataset_multi-label.csv') as csv_file:
    db_metadata = csv.reader(csv_file, delimiter=',')

    # Skip header row
    for row in db_metadata:
        msd_id_map[row[0]] = msd_id_map.get(row[0], []) + [row[2]]
        count += 1

# msd_id_map maps the amazon_ids to MSD_track_ids, to retrieve the actual information from the MSD dataset
len(msd_id_map.keys()), count

(31472, 147296)

In [5]:
import tqdm.notebook as tqdm

with open(mumu_dir + '/MuMu_dataset_multi-label.csv') as csv_file:
    db_metadata = csv.reader(csv_file, delimiter=',')
    
    msd_ids = set()
    
    for row in tqdm.tqdm(db_metadata):
        msd_ids.add(row[2])
        
len(msd_ids)

147296

In [6]:
msd_reviews = []

for review in db_review:
    for track_id in msd_id_map[review['amazon_id']]:
        obj = {}
        obj['amazon_id'] = review['amazon_id']
        obj['track_id'] = track_id
        obj['review_text'] = review['reviewText']
        obj['review_summary'] = review['summary']

        msd_reviews.append(obj)

len(msd_reviews)

2242154

In [8]:
import json
import lmdb

import time

from source.utils import *

SEQ_MAX_LEN = 64
root_dir='../msd/data/'

# 147296
env = lmdb.open('data/MSD/MSD_ID_to_Echonest.lmdb', subdir=False,
                map_size=1e12 )

msd_ids = list(msd_ids)

for track_id_list in tqdm(grouper(msd_ids[116000:], 1000, "0")):

    with env.begin(write=True) as txn: 
        
        for track_id in track_id_list:
            song_features = id_to_features(track_id, root_dir)
            if song_features is None:
                continue
            song_features = song_features.tolist()
            
            txn.put(track_id.encode(), json.dumps(song_features).encode())
            
env.stat()

KeyboardInterrupt: 

In [48]:
env.stat()

{'psize': 4096,
 'depth': 3,
 'branch_pages': 17,
 'leaf_pages': 1917,
 'overflow_pages': 8293330,
 'entries': 147295}

In [65]:
len(db_review), env.stat()

(447583,
 {'psize': 4096,
  'depth': 4,
  'branch_pages': 1611,
  'leaf_pages': 96799,
  'overflow_pages': 34685,
  'entries': 446214})

In [82]:
from random import sample

env = lmdb.open('data/MSD/MSD_ID_to_review_with_CLS.lmdb', subdir=False,
                map_size=1e12, )

def write_on_db(idx, obj):
    str_id = '{:08}'.format(idx).encode("ascii")
    with env.begin(write=True) as txn:
        txn.put(str_id, json.dumps(obj).encode())

idx = 0
#2242154
for review in tqdm(db_review):
    track_list = msd_id_map[review['amazon_id']]
    for track_id in track_list:

        true_obj = {'CLS': 1,
                    'review_text': review['reviewText'],
                    'summary': review['summary'],
                    'msd_id': track_id}
        
        # Samples a track not associated with this review
        # --> since there are 100k songs and just 10ish associated
        #     with each, it shouldn't be a problem of efficiency
        sampled_track_id = track_id
        while sampled_track_id in track_list or sampled_track_id == "MSD_track_id":
            sampled_track_id = sample(msd_ids, 1)[0]
        
        false_obj= {'CLS': 0,
                    'review_text': review['reviewText'],
                    'summary': review['summary'],
                    'msd_id': sampled_track_id}
        
        # Even indexes are for True_CLS objects, Odd for False_CLS 
        write_on_db(2*idx, true_obj)
        write_on_db(2*idx+1, false_obj)
        idx += 1
        
env.stat()

{'psize': 4096,
 'depth': 4,
 'branch_pages': 5660,
 'leaf_pages': 1162079,
 'overflow_pages': 375738,
 'entries': 4484308}

In [114]:
# x = [ track for review in db_review for track in msd_id_map[review['amazon_id']]]

# set(msd_ids).difference(set(x)), len(set(x))

env = lmdb.open('data/MSD/MSD_ID_to_review_with_CLS.lmdb', subdir=False, map_size=1e12)

bad_objs_list = []
bad_objs_ref = []

with env.begin(write=True) as txn:
    
    for key, value in tqdm(txn.cursor()):
        review_obj = json.loads(value)
        if "_track" in review_obj['msd_id']:
            bad_objs_list.append(key)
            for review in db_review:
                if review['reviewText'] == review_obj['review_text']:
                    bad_objs_ref.append(review)
                    
len(bad_objs_list)

0

In [115]:
for id, review in zip(bad_objs_list, bad_objs_ref):
    track_list = msd_id_map[review['amazon_id']]
    sampled_track_id = "MSD_track_id"
    while sampled_track_id in track_list or sampled_track_id == "MSD_track_id":
        sampled_track_id = sample(msd_ids, 1)[0]

    with env.begin(write=True) as txn:
        data = txn.get(id)
        review_obj = json.loads(data)
        review_obj['msd_id'] = sampled_track_id
        
        #txn.put(id, json.dumps(review_obj).encode())

In [7]:
id_list = []

for key,val in msd_id_map.items():
    id_list.extend(val)

id_list = list(set(id_list))

len(id_list)

147296

In [32]:
with open('json/msd_reviews.json', 'w') as outfile:
    json.dump(msd_reviews, outfile)

In [8]:
import tqdm.notebook as tqdm

data = []


for item in msd_reviews[:1]:
    print(item)

# for ip in tqdm.tqdm(id_list):
#     track = get_song_features(path)
#     data.append(track)

{'amazon_id': '1458389375', 'track_id': 'TRCQFGP12903CB68CA', 'review_text': 'Earth and Sky Dancing Music, intelligent on so many levels. Co-writers Costa/ St.John Ramirez deliver a blend of Indigenous Soul Music with R&B and Afro Latin Rhythms that is smooth, fun & deep. And with Justo Almario on horns - can you handle it? Oh Yea!', 'review_summary': 'Smooth, Fun & Deep -  And What A Voice!'}


In [9]:
for item in msd_reviews[:1]:
    print(item.keys())

dict_keys(['amazon_id', 'track_id', 'review_text', 'review_summary'])


In [10]:
len(msd_reviews)

2242154

In [11]:
import csv

id_to_file = {}
base_path = "../msd/mp3/MSD/audio/"

with open("../msd/mp3/MSD/dataset_annotation.csv", "r") as f:
    
    data = csv.reader(f)
    
    for row in data:
        if 'id' in row:
            continue
            
        name = row[1]
        path = base_path + name[0] + "/" + name[1] + "/" + name
        id_to_file[row[0]] = path
        
len(id_to_file.keys())

993175

In [43]:
import librosa
import lmdb
from source.io.parallel_processing import parallel_extract
import warnings
from source.io.pb_item import pb_Item
from source.sound_utils import get_mfcc

warnings.simplefilter("ignore")
file_path = "data/MSD/id_to_mfcc.bin"

# 147296
env = lmdb.open('data/MSD/MSD_ID_to_MFCC.lmdb', subdir=False,
                map_size=1e12 )

def process_id(id):
    
    id = id[:-1] # Remove last char or id that is not present in MoodAgent CSV
    
    if id not in id_to_file.keys():
        return
    
    path = id_to_file[id]
    
    try:
        mfcc = get_mfcc(path)
        pb_item = pb_Item(features=mfcc)
        
        with env.begin(write=True) as txn:
            txn.put(id.encode(), pb_item.get_pb_obj().SerializeToString())
                
    except KeyboardInterrupt as e:
        raise e
    except Exception as e:
        print(e)
        print("Item %s discarded"%row[0])

msd_ids = list(msd_ids)

parallel_extract(msd_ids, process_id)
    
env.stat()


Done


{'psize': 4096,
 'depth': 3,
 'branch_pages': 17,
 'leaf_pages': 1899,
 'overflow_pages': 16131379,
 'entries': 147295}

In [12]:
!ls -lh data/MSD/

total 102G
-rw-r--r--. 1 nobody nogroup  32G Jan 14 12:23 MSD_ID_to_Echonest.lmdb
-rw-r--r--. 1 nobody nogroup 8.0K Feb 18 15:25 MSD_ID_to_Echonest.lmdb-lock
-rw-r--r--. 1 nobody nogroup  62G Feb 18 21:10 MSD_ID_to_MFCC.lmdb
-rw-r--r--. 1 nobody nogroup 8.0K Feb 18 21:10 MSD_ID_to_MFCC.lmdb-lock
-rw-r--r--. 1 nobody nogroup 2.8G Jan 14 13:56 MSD_ID_to_review.lmdb
-rw-r--r--. 1 nobody nogroup 8.0K Feb 19 10:23 MSD_ID_to_review.lmdb-lock
-rw-r--r--. 1 nobody nogroup 5.9G Feb 20 09:23 MSD_ID_to_review_with_CLS.lmdb
-rw-r--r--. 1 nobody nogroup 8.0K Feb 20 09:24 MSD_ID_to_review_with_CLS.lmdb-lock


In [15]:
from random import sample
from tqdm.notebook import tqdm

csv_file = []
#2242154
for review in tqdm(db_review):
    track_list = msd_id_map[review['amazon_id']]
    for track_id in track_list:

        true_obj = {'CLS': 1,
                    'review_text': review['reviewText'],
                    'summary': review['summary'],
                    'msd_id': track_id}
        
        sampled_track_id = track_id
        while sampled_track_id in track_list or sampled_track_id == "MSD_track_id":
            sampled_track_id = sample(msd_ids, 1)[0]
        
        false_obj= {'CLS': 0,
                    'review_text': review['reviewText'],
                    'summary': review['summary'],
                    'msd_id': sampled_track_id}

        csv_file.append(true_obj)
        csv_file.append(false_obj)

KeyboardInterrupt: 

In [1]:
import pyspark
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType, IntegerType
from pyspark.sql import SparkSession

spark = SparkSession.builder \
         .master("local") \
         .appName("Word Count") \
         .getOrCreate()

schema = StructType([StructField("value", IntegerType(), True)])
my_list = [1, 2, 3, 4]

spark.createDataFrame(my_list, IntegerType()).show()

Py4JJavaError: An error occurred while calling o45.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 267, in main
    ("%d.%d" % sys.version_info[:2], version))
Exception: Python in worker has different version 2.7 than that in driver 3.6, PySpark cannot run with different minor versions.Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:456)
	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRunner.scala:592)
	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRunner.scala:575)
	at or